In [1]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda, Reshape, LSTM, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [2]:
tokenized_text, char_to_idx, idx_to_char = pickle.load(open('../data/processed-alice.pickle', 'rb'))
X, Y = pickle.load(open('../data/alice-x-y-data.pickle', 'rb'))

Y = np.reshape(Y, (X.shape[1], X.shape[0], X.shape[2]))

# Global Variables
m, Tx, n_values = X.shape
n_a = 64 # for LSTM with 64-dimensional hidden states

# Train the Model
First we train the model to get the proper weights.
Remember, we must declare the layers in our model as global variables so they
can be reused in the Text Generation step

In [3]:
# Reused Layer for Text Generation: Reshape, LSTM, Dense
reshapor = Reshape((1, n_values))
LSTM_cell = LSTM(n_a, return_state=True)
densor = Dense(n_values, activation='softmax')

In [4]:
def learn_to_talk(Tx, n_a, n_values):
    X = Input(shape=(Tx, n_values))
    
    # Define initial hidden state for LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    outputs = []
    for t in range(Tx):
        # Get t'th vector
        x = Lambda(lambda x: x[:,t,:])(X)
        # Reshape x to be (1,Tx)
        x = reshapor(x)
        # Perform 1 step of LSTM
        a, _, c = LSTM_cell(x)
        # apply densor to hidden state output of LSTM_cell
        out = densor(a)
        outputs.append(out)
    return Model([X, a0, c0], outputs)

In [5]:
model = learn_to_talk(Tx, n_a, n_values)

In [6]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

# a0 = np.random.rand(m, n_a)
# c0 = np.random.rand(m, n_a)

In [8]:
model.fit([X, a0, c0], list(Y), epochs=40)
# 5

Epoch 1/40
1031/1031 [==============================] - 168s 163ms/step - loss: 527.7057 - dense_1_loss: 3.7693 - dense_1_acc: 0.1038 - dense_1_acc_1: 0.1106 - dense_1_acc_2: 0.1319 - dense_1_acc_3: 0.0999 - dense_1_acc_4: 0.1057 - dense_1_acc_5: 0.0999 - dense_1_acc_6: 0.0980 - dense_1_acc_7: 0.0873 - dense_1_acc_8: 0.1125 - dense_1_acc_9: 0.1038 - dense_1_acc_10: 0.0912 - dense_1_acc_11: 0.1280 - dense_1_acc_12: 0.0960 - dense_1_acc_13: 0.1009 - dense_1_acc_14: 0.1048 - dense_1_acc_15: 0.1115 - dense_1_acc_16: 0.0931 - dense_1_acc_17: 0.0980 - dense_1_acc_18: 0.1018 - dense_1_acc_19: 0.1018 - dense_1_acc_20: 0.0892 - dense_1_acc_21: 0.1018 - dense_1_acc_22: 0.0912 - dense_1_acc_23: 0.0999 - dense_1_acc_24: 0.0980 - dense_1_acc_25: 0.1038 - dense_1_acc_26: 0.1018 - dense_1_acc_27: 0.1048 - dense_1_acc_28: 0.1804 - dense_1_acc_29: 0.1028 - dense_1_acc_30: 0.1057 - dense_1_acc_31: 0.1038 - dense_1_acc_32: 0.1038 - dense_1_acc_33: 0.0766 - dense_1_acc_34: 0.0970 - dense_1_acc_35: 0.0960 

  64/1031 [>.............................] - ETA: 4s - loss: 492.5090 - dense_1_loss: 3.5675 - dense_1_acc: 0.1719 - dense_1_acc_1: 0.0938 - dense_1_acc_2: 0.1406 - dense_1_acc_3: 0.1562 - dense_1_acc_4: 0.2188 - dense_1_acc_5: 0.1406 - dense_1_acc_6: 0.1406 - dense_1_acc_7: 0.1719 - dense_1_acc_8: 0.2656 - dense_1_acc_9: 0.1875 - dense_1_acc_10: 0.1875 - dense_1_acc_11: 0.2500 - dense_1_acc_12: 0.1250 - dense_1_acc_13: 0.2344 - dense_1_acc_14: 0.1875 - dense_1_acc_15: 0.2969 - dense_1_acc_16: 0.1719 - dense_1_acc_17: 0.1875 - dense_1_acc_18: 0.2188 - dense_1_acc_19: 0.1094 - dense_1_acc_20: 0.1719 - dense_1_acc_21: 0.2188 - dense_1_acc_22: 0.1719 - dense_1_acc_23: 0.1719 - dense_1_acc_24: 0.1562 - dense_1_acc_25: 0.1562 - dense_1_acc_26: 0.1719 - dense_1_acc_27: 0.2031 - dense_1_acc_28: 0.4531 - dense_1_acc_29: 0.1250 - dense_1_acc_30: 0.1719 - dense_1_acc_31: 0.1875 - dense_1_acc_32: 0.2812 - dense_1_acc_33: 0.2188 - dense_1_acc_34: 0.2031 - dense_1_acc_35: 0.2500 - dense_1_acc_36: 0

1031/1031 [==============================] - 5s 4ms/step - loss: 476.7405 - dense_1_loss: 3.4054 - dense_1_acc: 0.1920 - dense_1_acc_1: 0.1940 - dense_1_acc_2: 0.1920 - dense_1_acc_3: 0.1843 - dense_1_acc_4: 0.1988 - dense_1_acc_5: 0.2027 - dense_1_acc_6: 0.1920 - dense_1_acc_7: 0.1901 - dense_1_acc_8: 0.2502 - dense_1_acc_9: 0.1872 - dense_1_acc_10: 0.1969 - dense_1_acc_11: 0.2570 - dense_1_acc_12: 0.2241 - dense_1_acc_13: 0.1950 - dense_1_acc_14: 0.1950 - dense_1_acc_15: 0.2308 - dense_1_acc_16: 0.1959 - dense_1_acc_17: 0.2008 - dense_1_acc_18: 0.1872 - dense_1_acc_19: 0.1901 - dense_1_acc_20: 0.1872 - dense_1_acc_21: 0.1969 - dense_1_acc_22: 0.1988 - dense_1_acc_23: 0.1775 - dense_1_acc_24: 0.1930 - dense_1_acc_25: 0.1911 - dense_1_acc_26: 0.1920 - dense_1_acc_27: 0.2541 - dense_1_acc_28: 0.4316 - dense_1_acc_29: 0.1901 - dense_1_acc_30: 0.1882 - dense_1_acc_31: 0.1930 - dense_1_acc_32: 0.1959 - dense_1_acc_33: 0.1950 - dense_1_acc_34: 0.1940 - dense_1_acc_35: 0.1930 - dense_1_acc_3

  64/1031 [>.............................] - ETA: 4s - loss: 432.1412 - dense_1_loss: 3.1443 - dense_1_acc: 0.1875 - dense_1_acc_1: 0.2188 - dense_1_acc_2: 0.1250 - dense_1_acc_3: 0.0938 - dense_1_acc_4: 0.1562 - dense_1_acc_5: 0.2344 - dense_1_acc_6: 0.0938 - dense_1_acc_7: 0.1250 - dense_1_acc_8: 0.2344 - dense_1_acc_9: 0.2188 - dense_1_acc_10: 0.1719 - dense_1_acc_11: 0.2188 - dense_1_acc_12: 0.2188 - dense_1_acc_13: 0.2344 - dense_1_acc_14: 0.2188 - dense_1_acc_15: 0.2031 - dense_1_acc_16: 0.2188 - dense_1_acc_17: 0.2031 - dense_1_acc_18: 0.1875 - dense_1_acc_19: 0.1250 - dense_1_acc_20: 0.1250 - dense_1_acc_21: 0.2344 - dense_1_acc_22: 0.1406 - dense_1_acc_23: 0.1250 - dense_1_acc_24: 0.2188 - dense_1_acc_25: 0.3281 - dense_1_acc_26: 0.1250 - dense_1_acc_27: 0.2344 - dense_1_acc_28: 0.4844 - dense_1_acc_29: 0.1719 - dense_1_acc_30: 0.2188 - dense_1_acc_31: 0.1719 - dense_1_acc_32: 0.1562 - dense_1_acc_33: 0.1562 - dense_1_acc_34: 0.2500 - dense_1_acc_35: 0.2031 - dense_1_acc_36: 0

1031/1031 [==============================] - 5s 5ms/step - loss: 427.7508 - dense_1_loss: 3.0878 - dense_1_acc: 0.1920 - dense_1_acc_1: 0.1940 - dense_1_acc_2: 0.1920 - dense_1_acc_3: 0.1843 - dense_1_acc_4: 0.1988 - dense_1_acc_5: 0.2017 - dense_1_acc_6: 0.1920 - dense_1_acc_7: 0.1901 - dense_1_acc_8: 0.2502 - dense_1_acc_9: 0.1882 - dense_1_acc_10: 0.1969 - dense_1_acc_11: 0.2561 - dense_1_acc_12: 0.2241 - dense_1_acc_13: 0.1950 - dense_1_acc_14: 0.1950 - dense_1_acc_15: 0.2328 - dense_1_acc_16: 0.1959 - dense_1_acc_17: 0.2008 - dense_1_acc_18: 0.1872 - dense_1_acc_19: 0.1901 - dense_1_acc_20: 0.1872 - dense_1_acc_21: 0.1969 - dense_1_acc_22: 0.1988 - dense_1_acc_23: 0.1775 - dense_1_acc_24: 0.1930 - dense_1_acc_25: 0.1911 - dense_1_acc_26: 0.1920 - dense_1_acc_27: 0.2541 - dense_1_acc_28: 0.4316 - dense_1_acc_29: 0.1901 - dense_1_acc_30: 0.1882 - dense_1_acc_31: 0.1930 - dense_1_acc_32: 0.1959 - dense_1_acc_33: 0.1950 - dense_1_acc_34: 0.1940 - dense_1_acc_35: 0.1920 - dense_1_acc_3

...

1031/1031 [==============================] - 6s 5ms/step - loss: 415.9082 - dense_1_loss: 3.0595 - dense_1_acc: 0.1920 - dense_1_acc_1: 0.1940 - dense_1_acc_2: 0.1920 - dense_1_acc_3: 0.1843 - dense_1_acc_4: 0.1988 - dense_1_acc_5: 0.2017 - dense_1_acc_6: 0.1920 - dense_1_acc_7: 0.1901 - dense_1_acc_8: 0.2502 - dense_1_acc_9: 0.1882 - dense_1_acc_10: 0.1969 - dense_1_acc_11: 0.2561 - dense_1_acc_12: 0.2241 - dense_1_acc_13: 0.1950 - dense_1_acc_14: 0.1950 - dense_1_acc_15: 0.2328 - dense_1_acc_16: 0.1959 - dense_1_acc_17: 0.2008 - dense_1_acc_18: 0.1872 - dense_1_acc_19: 0.1901 - dense_1_acc_20: 0.1872 - dense_1_acc_21: 0.1969 - dense_1_acc_22: 0.1988 - dense_1_acc_23: 0.1775 - dense_1_acc_24: 0.1930 - dense_1_acc_25: 0.1911 - dense_1_acc_26: 0.1920 - dense_1_acc_27: 0.2541 - dense_1_acc_28: 0.4316 - dense_1_acc_29: 0.1901 - dense_1_acc_30: 0.1882 - dense_1_acc_31: 0.1930 - dense_1_acc_32: 0.1959 - dense_1_acc_33: 0.1950 - dense_1_acc_34: 0.1940 - dense_1_acc_35: 0.1920 - dense_1_acc_3

  64/1031 [>.............................] - ETA: 5s - loss: 416.6588 - dense_1_loss: 3.1067 - dense_1_acc: 0.1406 - dense_1_acc_1: 0.2031 - dense_1_acc_2: 0.1406 - dense_1_acc_3: 0.2031 - dense_1_acc_4: 0.2031 - dense_1_acc_5: 0.1250 - dense_1_acc_6: 0.1562 - dense_1_acc_7: 0.1719 - dense_1_acc_8: 0.3438 - dense_1_acc_9: 0.1562 - dense_1_acc_10: 0.1562 - dense_1_acc_11: 0.2656 - dense_1_acc_12: 0.1875 - dense_1_acc_13: 0.2812 - dense_1_acc_14: 0.1250 - dense_1_acc_15: 0.2812 - dense_1_acc_16: 0.1250 - dense_1_acc_17: 0.0781 - dense_1_acc_18: 0.1562 - dense_1_acc_19: 0.1094 - dense_1_acc_20: 0.1719 - dense_1_acc_21: 0.1719 - dense_1_acc_22: 0.2188 - dense_1_acc_23: 0.1719 - dense_1_acc_24: 0.2969 - dense_1_acc_25: 0.1562 - dense_1_acc_26: 0.1719 - dense_1_acc_27: 0.2500 - dense_1_acc_28: 0.4688 - dense_1_acc_29: 0.1406 - dense_1_acc_30: 0.1875 - dense_1_acc_31: 0.2188 - dense_1_acc_32: 0.2812 - dense_1_acc_33: 0.2656 - dense_1_acc_34: 0.1250 - dense_1_acc_35: 0.2188 - dense_1_acc_36: 0

1031/1031 [==============================] - 6s 5ms/step - loss: 415.8896 - dense_1_loss: 3.0599 - dense_1_acc: 0.1920 - dense_1_acc_1: 0.1940 - dense_1_acc_2: 0.1920 - dense_1_acc_3: 0.1843 - dense_1_acc_4: 0.1988 - dense_1_acc_5: 0.2017 - dense_1_acc_6: 0.1920 - dense_1_acc_7: 0.1901 - dense_1_acc_8: 0.2502 - dense_1_acc_9: 0.1882 - dense_1_acc_10: 0.1969 - dense_1_acc_11: 0.2561 - dense_1_acc_12: 0.2241 - dense_1_acc_13: 0.1950 - dense_1_acc_14: 0.1950 - dense_1_acc_15: 0.2328 - dense_1_acc_16: 0.1959 - dense_1_acc_17: 0.2008 - dense_1_acc_18: 0.1872 - dense_1_acc_19: 0.1901 - dense_1_acc_20: 0.1872 - dense_1_acc_21: 0.1969 - dense_1_acc_22: 0.1988 - dense_1_acc_23: 0.1775 - dense_1_acc_24: 0.1930 - dense_1_acc_25: 0.1911 - dense_1_acc_26: 0.1920 - dense_1_acc_27: 0.2541 - dense_1_acc_28: 0.4316 - dense_1_acc_29: 0.1901 - dense_1_acc_30: 0.1882 - dense_1_acc_31: 0.1930 - dense_1_acc_32: 0.1959 - dense_1_acc_33: 0.1950 - dense_1_acc_34: 0.1940 - dense_1_acc_35: 0.1920 - dense_1_acc_3

  64/1031 [>.............................] - ETA: 5s - loss: 414.3233 - dense_1_loss: 2.9261 - dense_1_acc: 0.2031 - dense_1_acc_1: 0.2031 - dense_1_acc_2: 0.1094 - dense_1_acc_3: 0.1562 - dense_1_acc_4: 0.2656 - dense_1_acc_5: 0.1562 - dense_1_acc_6: 0.2031 - dense_1_acc_7: 0.1406 - dense_1_acc_8: 0.2500 - dense_1_acc_9: 0.1250 - dense_1_acc_10: 0.2188 - dense_1_acc_11: 0.2969 - dense_1_acc_12: 0.1250 - dense_1_acc_13: 0.2188 - dense_1_acc_14: 0.1719 - dense_1_acc_15: 0.2344 - dense_1_acc_16: 0.1406 - dense_1_acc_17: 0.1562 - dense_1_acc_18: 0.2500 - dense_1_acc_19: 0.2344 - dense_1_acc_20: 0.1875 - dense_1_acc_21: 0.1875 - dense_1_acc_22: 0.1094 - dense_1_acc_23: 0.2500 - dense_1_acc_24: 0.1875 - dense_1_acc_25: 0.1250 - dense_1_acc_26: 0.1406 - dense_1_acc_27: 0.2344 - dense_1_acc_28: 0.4844 - dense_1_acc_29: 0.2656 - dense_1_acc_30: 0.2031 - dense_1_acc_31: 0.2500 - dense_1_acc_32: 0.2188 - dense_1_acc_33: 0.1875 - dense_1_acc_34: 0.2188 - dense_1_acc_35: 0.1719 - dense_1_acc_36: 0

1031/1031 [==============================] - 6s 5ms/step - loss: 415.8760 - dense_1_loss: 3.0598 - dense_1_acc: 0.1920 - dense_1_acc_1: 0.1940 - dense_1_acc_2: 0.1920 - dense_1_acc_3: 0.1843 - dense_1_acc_4: 0.1988 - dense_1_acc_5: 0.2017 - dense_1_acc_6: 0.1920 - dense_1_acc_7: 0.1901 - dense_1_acc_8: 0.2502 - dense_1_acc_9: 0.1882 - dense_1_acc_10: 0.1969 - dense_1_acc_11: 0.2561 - dense_1_acc_12: 0.2241 - dense_1_acc_13: 0.1950 - dense_1_acc_14: 0.1950 - dense_1_acc_15: 0.2328 - dense_1_acc_16: 0.1959 - dense_1_acc_17: 0.2008 - dense_1_acc_18: 0.1872 - dense_1_acc_19: 0.1901 - dense_1_acc_20: 0.1872 - dense_1_acc_21: 0.1969 - dense_1_acc_22: 0.1988 - dense_1_acc_23: 0.1775 - dense_1_acc_24: 0.1930 - dense_1_acc_25: 0.1911 - dense_1_acc_26: 0.1920 - dense_1_acc_27: 0.2541 - dense_1_acc_28: 0.4316 - dense_1_acc_29: 0.1901 - dense_1_acc_30: 0.1882 - dense_1_acc_31: 0.1930 - dense_1_acc_32: 0.1959 - dense_1_acc_33: 0.1950 - dense_1_acc_34: 0.1940 - dense_1_acc_35: 0.1920 - dense_1_acc_3

# Use Trained Weights for Text Generation

In [9]:
def one_hot(softmax_v):
    best_idxs = K.argmax(softmax_v, axis=-1)
    return K.one_hot([best_idxs], num_classes=n_values)

def generate_text(Ty = 140):
    x0 = Input(shape=(Tx, n_values))
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0
    outputs = []
    
    for t in range(Ty):
        # Get new activation and cell state
        a, _ , c = LSTM_cell(x, initial_state=[a, c])
        # Use Dense layer to get softmax output
        out = densor(a)
        outputs.append(out)
        
        x = Lambda(one_hot)(out)
    
    return Model([x0, a0, c0], outputs)
    

In [10]:
text_model = generate_text()

In [11]:
x_initializer = np.zeros((1, Tx, n_values))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

# x_initializer = np.random.rand(1, 1, n_values)
# a_initializer = np.random.rand(1, n_a)
# c_initializer = np.random.rand(1, n_a)

In [12]:
def predict_and_sample(x_initializer, a_initializer, c_initializer):
    pred = text_model.predict([x_initializer, a_initializer, c_initializer])
    indices = np.argmax(pred, axis=-1)
    results = to_categorical(indices, num_classes=n_values)
    return results, indices

In [13]:
results, indices = predict_and_sample(x_initializer, a_initializer, c_initializer)

In [14]:
new_chars = [idx_to_char[str(i)] for i in np.ndarray.flatten(indices)]

In [15]:
''.join(new_chars)

'                                                                                                                                            '

In [16]:
indices

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    